Import packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Prepare BANet metrics

In [ ]:
metrics = pd.read_csv('banet_metrics.csv')

Show BANet results

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,8))

# Plot validation L1-Loss
ax1.plot(metrics['Train Loss'], 'b', linewidth=2)
ax1.plot(metrics['Test Loss'], 'r', linewidth=2)
ax1.legend(['training', 'validation'], fontsize=14)
ax1.set_xlabel('epoch', fontsize=16)
ax1.set_ylabel('L1-Loss', fontsize=16)
ax1.tick_params(axis='both', which='major', labelsize=12)
ax1.grid()

# Plot validation SILog Loss
ax2.plot(metrics['Train SILog'], 'b', linewidth=2)
ax2.plot(metrics['Test SILog'], 'r', linewidth=2)
ax2.legend(['training', 'validation'], fontsize=14)
ax2.set_xlabel('epoch', fontsize=16)
ax2.set_ylabel('SILog Loss', fontsize=16)
ax2.tick_params(axis='both', which='major', labelsize=12)
ax2.grid()